# Tutorial notebook for LCA-Modeller

Welcome to the *LCA-Modeller* library tutorial! *LCA-Modeller* offers a streamlined interface designed to facilitate the creation of **parametric LCA models** with **prospective capabilities**. It builds on the open-source libraries [*lca-algebraic*](https://lca-algebraic.readthedocs.io/) and [*premise*](https://premise.readthedocs.io/), so having a basic understanding of these tools, while not essential, is recommended.

The core functionality of *LCA-Modeller* revolves around reading a user-provided configuration file that defines the LCA model. From this configuration, *LCA-Modeller* generates a parametric LCA model, which can be evaluated for any parameter values using *lca-algebraic*'s built-in functions.
If prospective scenarios are provided, *premise* is used to adapt the EcoInvent database to future conditions. The parametric LCA model then selects the appropriate prospective databases based on the user-specified evaluation year.

In [ ]:
# Import libraries
from lca_modeller.io.configuration import LCAProblemConfigurator
import lca_algebraic as agb
import brightway2 as bw
import numpy as np

# Add data folder to path
import sys
import os.path as pth
sys.path.insert(0, '../..')
DATA_FOLDER = './data'

## 1. A basic example

In [ ]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'basic_example.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

In [ ]:
# Provide values for parameters
parameters = {
    'a': 1.0,
    'b': 0.7,
    'c': 1.2,
    'd': 10.0,
}

# Compute impacts. The first call takes some time (compilation of the LCA functions) but next ones are much faster.
df = agb.compute_impacts(
    model,
    methods,
    **parameters
)
df

In [ ]:
# Compute impacts for several values at once
parameters = {
    'a': [1.0, 1.2, 1.4],  # list of values
    'b': [0.7, 0.5, 0.2],  # all lists must be of the same size
    'c': 1.2,  # constant value
    'd': 10.0,
}
agb.compute_impacts(
    model,
    methods,
    **parameters
)

In [ ]:
# Another example
parameters = {
    'a': np.linspace(1.0, 2.0, 50),
    'b': np.linspace(0.7, 0.0, 50),
    'c': 1.2,
    'd': 10.0,
}
agb.compute_impacts(
    model,
    methods,
    **parameters
)

## 2. Get impacts by phase or subsystem

In [ ]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'basic_example_split_impacts.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

In [ ]:
# Provide values for parameters
parameters = {
    'a': 1.0,
    'b': 0.7,
    'c': 1.2,
    'd': 10.0,
}

# Compute impacts and provide results by subsystem 
agb.compute_impacts(
    model,
    methods,
    axis='subsystem',
    **parameters
)

## 3. Prospective LCA

Warning: depending on the IAM, the locations might be changed compared to the original ecoinvent database.

In [ ]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'prospective_lca.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

In [ ]:
# List databases installed in the project
bw.databases

In [ ]:
# Running a project with premise will automatically add the parameter 'year' to the model to enable the user to select the year of evaluation.
parameters = {
    'year': [2020, 2025, 2030, 2035, 2040],  # if the year falls between two ecoinvent datasets (e.g. 2020 and 2030), the result will be a linear interpolation of the two
    'a': [1.0, 1.2, 1.4, 1.6, 1.8],
    'b': [0.7, 0.5, 0.2, 0.1, 0.0],
    'c': 1.2,
    'd': 10.0,
}
agb.compute_impacts(
    model,
    methods,
    **parameters
)

## 4. Advanced examples

### Reference a previously defined activity and modify existing activities

In [ ]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'advanced_example.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

In [ ]:
# Running a project with premise will automatically add the parameter 'year' to the model to enable the user to select the year of evaluation.
parameters = {
    'year': [2020, 2025, 2030, 2035, 2040],  # if the year falls between two ecoinvent datasets (e.g. 2020 and 2030), the result will be a linear interpolation of the two
    'a': [1.0, 1.2, 1.4, 1.6, 1.8],
    'b': [0.7, 0.5, 0.2, 0.1, 0.0],
    'c': 1.2,
    'd': 10.0,
    'p': [0.3, 0.2, 0.15, 0.12, 0.08]
}
agb.compute_impacts(
    model,
    methods,
    **parameters
)

## 5. Define your own LCIA methods

In [ ]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'custom_lcia_methods.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

In [ ]:
# The custom method is automatically added to the list
methods

In [ ]:
# Compute impacts with new method
parameters = {
    'a': [1.0, 1.2, 1.4, 1.6, 1.8],
    'b': [0.7, 0.5, 0.2, 0.1, 0.0],
    'c': 1.2,
    'd': 10.0,
}

agb.compute_impacts(
    model,
    methods,
    **parameters
)

# 6. Declare metadata for parameters
For example, declare min/max values and distribution type for each parameter to run a Monte Carlo using lca-algebraic's capabilities

In [ ]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'metadata_example.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

In [ ]:
agb.list_parameters()

In [ ]:
agb.incer_stochastic_dashboard(model, methods, n=1024)